# Twitter sentiment analysis using AutoML

In [1]:
!pip install requests tabulate future matplotlib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 1.3 MB/s eta 0:00:00a 0:00:01
  Using cached tabulate-0.9.0-py3-none-any.whl (35 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 840.9/840.9 kB 8.4 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 11.9 MB/s eta 0:00:0000:0100:01
  Using cached certifi-2022.12.7-py3-none-any.whl (155 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.8/198.8 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 12.4 MB/s eta 0:00:00a 0:00:01
  Using cached fonttools-4.38.0-py3-none-any.whl (965 kB)
  Using cached cycler-0.11.0-py3-none-any.whl (6.4 kB)
  Using cached kiwisolver-1.4.4-cp310-cp310-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (1.6 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.3/300.3 kB 11.2 MB/s eta 0:00:00
  Using cached pyparsing-3.0.9-

In [1]:
!pip install -f http://h2o-release.s3.amazonaws.com/h2o/latest_stable_Py.html h2o

Looking in links: http://h2o-release.s3.amazonaws.com/h2o/latest_stable_Py.html

[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: pip install --upgrade pip


In [2]:
import h2o
from h2o.automl import H2OAutoML

h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.17" 2022-10-18; OpenJDK Runtime Environment (build 11.0.17+8-post-Ubuntu-1ubuntu222.04); OpenJDK 64-Bit Server VM (build 11.0.17+8-post-Ubuntu-1ubuntu222.04, mixed mode, sharing)
  Starting server from /home/barteus/Work/DSV/kubeflow-examples/venv2/lib/python3.10/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmpt823bdxl
  JVM stdout: /tmp/tmpt823bdxl/h2o_barteus_started_from_python.out
  JVM stderr: /tmp/tmpt823bdxl/h2o_barteus_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,01 secs
H2O_cluster_timezone:,Europe/Warsaw
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.40.0.1
H2O_cluster_version_age:,18 days
H2O_cluster_name:,H2O_from_python_barteus_9iyi2m
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,7.770 Gb
H2O_cluster_total_cores:,8
H2O_cluster_allowed_cores:,8
H2O_cluster_status:,"locked, healthy"


In [4]:
df = h2o.import_file("https://raw.githubusercontent.com/choas/h2o-titanic/master/data/train.csv")

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [5]:
nrows_prev = df.nrows
not_empty_embarked = ~df["Embarked"].isna()
df_e = df[not_empty_embarked]
not_empty_age = df_e["Age"] >= 0
df = df_e[not_empty_age]
print("%d rows removed" % (nrows_prev - df.nrows))

179 rows removed


In [6]:
# filter unknown Age and split 70% train and 30% test
train, test = df[df["Age"] >= 0].split_frame(ratios=[.7])

In [7]:
# Identify predictors and response
x = train.columns
y = "Survived"
x.remove(y)

# remove some columns
x.remove("PassengerId")
x.remove("Name")
x.remove("Ticket")
x.remove("Cabin")

In [8]:
# For binary classification, response should be a factor
train[y] = train[y].asfactor()
test[y] = test[y].asfactor()

train["Pclass"] = train["Pclass"].asfactor()
test["Pclass"] = test["Pclass"].asfactor()

In [9]:
train.summary()

/tmp/ipykernel_26606/3201333431.py:1: H2ODeprecationWarning: ``summary()`` is deprecated, please use ``show_summary()`` or ``get_summary()`` instead
  train.summary()


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
type,int,enum,enum,string,enum,real,int,int,int,real,enum,enum
mins,1.0,,,NaN,,0.42,0.0,0.0,693.0,0.0,,
mean,450.40718562874224,,,NaN,,29.549241516966077,0.5449101796407183,0.41916167664670684,287598.5382513659,35.105114570858255,,
maxs,891.0,,,NaN,,80.0,5.0,6.0,3101296.0,512.3292,,
sigma,256.93821410655045,,,NaN,,14.384895182918536,0.9800403266785341,0.8740435319870421,537962.9089762091,56.60094303009492,,
zeros,0,,,0,,0,327,374,0,5,,
missing,0,0,0,0,0,0,0,0,135,0,375,0
0,1.0,0,3,"Braund, Mr. Owen Harris",male,22.0,1.0,0.0,nan,7.25,,S
1,2.0,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Thayer)",female,38.0,1.0,0.0,nan,71.2833,C85,C
2,3.0,1,3,"Heikkinen, Miss. Laina",female,26.0,0.0,0.0,nan,7.925,,S


In [10]:
aml = H2OAutoML(max_runtime_secs = 120)
aml.train(x = x, y = y, training_frame = train, validation_frame = test)

AutoML progress: |
11:03:17.962: User specified a validation frame with cross-validation still enabled. Please note that the models will still be validated using cross-validation only, the validation frame will be used to provide purely informative validation metrics on the trained models.

███████████████████████████████████████████████████████████████| (done) 100%


key,value
Stacking strategy,cross_validation
Number of base models (used / total),6/6
# GBM base models (used / total),1/1
# XGBoost base models (used / total),1/1
# DRF base models (used / total),2/2
# GLM base models (used / total),1/1
# DeepLearning base models (used / total),1/1
Metalearner algorithm,GLM
Metalearner fold assignment scheme,Random
Metalearner nfolds,5


In [11]:
lb = aml.leaderboard
lb.head(rows=lb.nrows)

model_id,auc,logloss,aucpr,mean_per_class_error,rmse,mse
StackedEnsemble_BestOfFamily_3_AutoML_1_20230224_110317,0.846133,0.459356,0.829293,0.203176,0.382731,0.146483
GBM_grid_1_AutoML_1_20230224_110317_model_12,0.845606,0.465041,0.827992,0.220166,0.386294,0.149223
GBM_grid_1_AutoML_1_20230224_110317_model_9,0.844734,0.458622,0.831179,0.208532,0.382822,0.146552
StackedEnsemble_BestOfFamily_4_AutoML_1_20230224_110317,0.844405,0.461446,0.824273,0.20464,0.383404,0.146999
GBM_2_AutoML_1_20230224_110317,0.844232,0.464513,0.822217,0.213387,0.384974,0.148205
XGBoost_lr_search_selection_AutoML_1_20230224_110317_select_grid_model_3,0.844084,0.474957,0.824804,0.212613,0.390422,0.152429
StackedEnsemble_AllModels_2_AutoML_1_20230224_110317,0.843772,0.46338,0.825211,0.203489,0.384146,0.147568
GBM_grid_1_AutoML_1_20230224_110317_model_13,0.843015,0.467719,0.811182,0.221129,0.387232,0.149949
GBM_grid_1_AutoML_1_20230224_110317_model_8,0.842447,0.468423,0.827004,0.225942,0.388615,0.151022
GBM_grid_1_AutoML_1_20230224_110317_model_7,0.841682,0.471494,0.824718,0.219475,0.390558,0.152535


In [13]:
preds = aml.leader.predict(test)
preds

stackedensemble prediction progress: |███████████████████████████████████████████| (done) 100%
stackedensemble prediction progress: |███████████████████████████████████████████| (done) 100%


predict,p0,p1
0,0.931376,0.0686239
0,0.733829,0.266171
1,0.412671,0.587329
1,0.329067,0.670933
1,0.275716,0.724284
0,0.940674,0.0593263
1,0.4401,0.5599
0,0.887337,0.112663
1,0.296694,0.703306
0,0.87071,0.12929


In [15]:
aml.leader.params

{'model_id': {'default': None,
  'actual': {'__meta': {'schema_version': 3,
    'schema_name': 'ModelKeyV3',
    'schema_type': 'Key<Model>'},
   'name': 'StackedEnsemble_BestOfFamily_3_AutoML_1_20230224_110317',
   'type': 'Key<Model>',
   'URL': '/3/Models/StackedEnsemble_BestOfFamily_3_AutoML_1_20230224_110317'},
  'input': None},
 'training_frame': {'default': None,
  'actual': {'__meta': {'schema_version': 3,
    'schema_name': 'FrameKeyV3',
    'schema_type': 'Key<Frame>'},
   'name': 'AutoML_1_20230224_110317_training_py_12_sid_b7bb',
   'type': 'Key<Frame>',
   'URL': '/3/Frames/AutoML_1_20230224_110317_training_py_12_sid_b7bb'},
  'input': {'__meta': {'schema_version': 3,
    'schema_name': 'FrameKeyV3',
    'schema_type': 'Key<Frame>'},
   'name': 'AutoML_1_20230224_110317_training_py_12_sid_b7bb',
   'type': 'Key<Frame>',
   'URL': '/3/Frames/AutoML_1_20230224_110317_training_py_12_sid_b7bb'}},
 'response_column': {'default': None,
  'actual': {'__meta': {'schema_version': 3

In [21]:
model_path = h2o.save_model(model=aml.leader, path="./mymodel", force=True)

In [22]:
h2o.load_model(model_path)

key,value
Stacking strategy,cross_validation
Number of base models (used / total),6/6
# GBM base models (used / total),1/1
# XGBoost base models (used / total),1/1
# DRF base models (used / total),2/2
# GLM base models (used / total),1/1
# DeepLearning base models (used / total),1/1
Metalearner algorithm,GLM
Metalearner fold assignment scheme,Random
Metalearner nfolds,5
